# Basics of spike train analysis

This is the first part of the course. 

**You will learn to:** 
- Explore spike trains and analyze their statistics.
- Plot stuff.

Let's first import the packages we are going to use, and set up some plotting parameters.

In [ ]:
# import packages
import numpy as np
import matplotlib.pyplot as plt
import spkbasic
from os import listdir
from os.path import isfile, join

%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 6.0) # set default size of plots

## 1 - Loading and plotting spike trains

Let's begin by reading in spike times in numpy arrays

### Loading single and multiple spike trains

In [ ]:
example_spikes_path='example_spikes.txt'
spike_times = np.loadtxt(example_spikes_path)

The file above contains the spike train of a single neuron in s.

In [ ]:
print(spike_times)

**Exercise:** Since we are going to use loading of spike trains often, we will make a function out of it. This exercise should also test your basic understanding of functions. Try to fill `loadSpikeTrain()`:

In [ ]:
def loadSpikeTrain(pathToSpikeTrain):
    '''
    Inputs:
            pathToSpikeTrain: string
    Outputs: 
            spike_times: numpy array
    '''
    
    ### START CODE HERE ### (approx. 3 lines)
    spike_file = open(pathToSpikeTrain)
    spike_times = np.array([float(line) for line in spike_file])
    spike_file.close()
    ### END CODE HERE ###
    
    return spike_times

In [ ]:
print(loadSpikeTrain('spike_trains/18_SP_C203.txt'))

Expected output: 
[  0.5766   2.8239   4.5523 ... 481.387  482.4371 482.4677]

It is also useful to load multiple spike trains in our workspace. One can stack them in a list.


**Exercise:** 

*Hint:* Make use of `np.loadtxt()` that we used above!

In [ ]:
def loadSpikeTrainsToList(list_of_paths):
    '''
    Inputs:
            list_of_paths: list of strings
    Outputs: 
            list_of_spikes: numpy array
    '''
    
    ### START CODE HERE ### (approx. 3 lines)
    list_of_spikes=[np.loadtxt(spath) for spath in list_of_paths]
    ### END CODE HERE ###
    
    return list_of_spikes

In [ ]:
mypath='spike_trains/'
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
print('Number of spike trains: '+ str(len(onlyfiles)))
print(loadSpikeTrainsToList(onlyfiles)[1])

Expected output:

[  0.5766   2.8239   4.5523 ... 481.387  482.4371 482.4677]

### Raster plots for spike train visualization

An useful function for plotting spike trains is eventplot

In [ ]:
plt.eventplot(spike_times)
#plt.plot(spike_times,np.zeros(spike_times.shape),'|', markersize=100)
plt.xlim([0,15])
plt.xlabel('Time (s)')

It is possible to use eventplot to stack spike trains on top of one another. Can you plot the spike trains of five neurons?

In [ ]:
mypath='spike_trains/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

spks=loadSpikeTrain(mypath+onlyfiles[0])
ii=0
all_trains=[]
for pathfile in onlyfiles[:5]:
    spks=loadSpikeTrain(mypath+pathfile)
    all_trains.append(spks)


In [ ]:
def plotMultipleTrains(spkTrainList):
    ### START CODE HERE ### (approx. 4 lines)
    
    plt.eventplot(all_trains)
    plt.xlim([0,25])
    ### END CODE HERE ###
    return;

In [ ]:
plotMultipleTrains(all_trains)

In [ ]:
sn = np.array(all_trains)
sn.shape

In [ ]:
mylist = []
mylist.append(spks)
mylist.append(all_trains[2])
len(mylist)
for i in mylist:
    print(len(i))

myarr = np.array(mylist)
myarr.shape
plt.eventplot(myarr)
plt.show()


Although spike times are very intuitive and easy to handle, we can run to problems. For that reason, we will introduce an alternative representation of spike trains.

## 2 - Binning spike trains

Spike trains almost never contain the same number of events. This fact makes their manipulation for plotting and analysis harder.

### Spike trains as discrete events

You can bin the spike trains of multiple neurons at once.

In [ ]:
bins=np.arange(0,15,0.01)
finds=np.digitize(spike_times, bins)
frate = [spike_times[finds == i] for i in range(1, len(bins))]
a = np.bincount(finds)
plt.plot(bins, a[:-1])
plt.show()

plt.imshow(a[None, :])


### Firing rates

**Exercise:** Bin the spike times in 10 ms bins.

**Exercise:** Did the drug significantly alter the firing rate?

## 3 - Spike train statistics

By looking at statistical properties of spike trains, we can extract useful information.

### Interspike interval histogram

Below, you see the interspike interval histogram of a single neuron:

In [ ]:
plt.hist(np.diff(spike_times)*1e3,np.linspace(0,30,100));

**Exercise:** Try to reproduce such a histogram by filling `plotIsiHistogram()` below. The functions `np.diff` and `plt.hist` might help you

In [ ]:
def plotIsiHistogram(spiketrain):
    ### START CODE HERE ### (approx. 2 lines)
    plt.hist(np.diff(spiketrain)*1e3,np.linspace(0,30,100));
    ### END CODE HERE ###

In [ ]:
plotIsiHistogram(spike_times)

**Exercise:** Estimate the % of intervals that are below 2 ms by filling in `percentIntervals(spike_train)`

In [ ]:
def percentIntervals(spiketrain,timeval):
    ### START CODE HERE ### (approx. 2 lines)
    spdiffs=np.diff(spiketrain)*1e3;
    pint=np.sum(spdiffs<timeval)/np.size(spiketrain)
    ### END CODE HERE ###
    return pint

In [ ]:
print("ISIs below 2 ms are " + str(100*percentIntervals(spike_times,2)) + "%")

**Expected output:**
1.15 %

### Autocorrelogram

In [ ]:
cres=np.correlate(spike_times,spike_times,mode='full')
plt.plot(cres)

### Crosscorrelogram